In [61]:
import matplotlib.pyplot as plt
import json
import glob
import os
import numpy as np
from config import problem_configs

In [62]:
#///////// Config //////////
FILEDIRECTORY = "isingBatches"

In [63]:
def load_file_into_dict(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

In [64]:
for problem_name, config in problem_configs.items():
        print(f"--- Processing class: {problem_name} ---")

        file_slug = config['file_slug']
        search_pattern = os.path.join(FILEDIRECTORY, f"MERGED*{file_slug}.json")
        distributionFile = glob.glob(search_pattern)[0]
        search_pattern = os.path.join(FILEDIRECTORY, f"solved*{file_slug}.json")
        exactSolutionsFile = glob.glob(search_pattern)[0]
        print(f'File containing distribution = {distributionFile}, File containing exact solutions = {exactSolutionsFile}')

        sampledResultsData = load_file_into_dict(distributionFile)
        exactSolutions = load_file_into_dict(exactSolutionsFile)

        qaoaSuccessCount = 0

        for run_instance in sampledResultsData["results"]:
                instance_id = run_instance["instance_id"]

                # The 'sampled_distribution' is now a sorted list of lists.
                # The most probable solution is the first item in the list.
                # e.g., ["010100001", 39]. We only need the bitstring, which is at index [0].
                most_probable_bitstring = run_instance["sampled_distribution"][0][0]

                # The original script reversed the string, so we preserve that logic.
                mostProbableSolution = most_probable_bitstring[::-1]

                # Find the corresponding exact solutions for this instance
                correctSolutionsIsing = []
                for exact_item in exactSolutions:
                        # Compare the integer instance_id directly
                        if exact_item["instance_id"] == instance_id:
                                correctSolutionsIsing = exact_item["solutions"]
                                # Optimization: once found, no need to continue searching
                                break 
                
                # This logic for converting from Ising to binary strings remains the same
                correctSolutionsBinary = []
                for ising_string in correctSolutionsIsing:
                        # 0s and 1s may seem mixed up here, but it's because the qubit state |0> 
                        # corresponds to the Z-eigenvalue of +1, and vice-versa.
                        binary_string = ising_string.replace('-1', '1').replace('+1', '0').replace(',', '')
                        correctSolutionsBinary.append(binary_string)
                # Check if the QAOA result is one of the valid ground states
                if mostProbableSolution in correctSolutionsBinary:
                        qaoaSuccessCount += 1

 
        # Print the final total success count
        print(f"Total QAOA Success Count: {qaoaSuccessCount} / {len(sampledResultsData['results'])}")

--- Processing class: TSP ---
File containing distribution = isingBatches/MERGED_QAOA_results_TSP_9q_.json, File containing exact solutions = isingBatches/solved_batch_Ising_data_TSP_9q_.json
Total QAOA Success Count: 35 / 100
--- Processing class: Knapsack ---
File containing distribution = isingBatches/MERGED_QAOA_results_Knapsack_6_items_9q_.json, File containing exact solutions = isingBatches/solved_batch_Ising_data_Knapsack_6_items_9q_.json
Total QAOA Success Count: 2 / 100
--- Processing class: MinimumVertexCover ---
File containing distribution = isingBatches/MERGED_QAOA_results_MinimumVertexCover_9q_.json, File containing exact solutions = isingBatches/solved_batch_Ising_data_MinimumVertexCover_9q_.json
Total QAOA Success Count: 4 / 100
